In [ ]:
def featuring(dataserie): # 6 feat
    return [np.mean(dataserie),
        np.var(dataserie),
        np.std(dataserie),
        sp.stats.skew(dataserie),
        sp.stats.kurtosis(dataserie),
        # sp.stats.gmean(dataserie) #cannot be applied to negatives,
        # sp.stats.hmean(dataserie) #cannot be applied to negatives,
        sp.stats.iqr(dataserie),
        # sp.stats.entropy(dataserie) #inf ocurrs
           ]


def multibandfeat(dataseries):
    feat = []
    for i in dataseries:  # 8
        for j in i:  # 4
            feat += featuring(j)  # 6
    return feat

In [44]:
def prespeller(method){
     if method == "BatchPhasePower":
        phaseData = np.array([np.unwrap(np.angle(hilbert(i))) for i in bandpassData])
        powerData = np.array([np.abs(hilbert(i)) for i in bandpassData])
        return (phaseData,powerData)

    elif method == "FrequenzySeperated":
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        phaseData = np.array([np.array([ np.unwrap(np.angle(hilbert(i))) for i in j]) for j in KaiserData])
        powerData = np.array([np.array([ np.abs(hilbert(i)) for i in j]) for j in KaiserData])
        return (KaiserData,phaseData,powerData)
    elif method == "Alpha_theta":
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        return(KaiserData[:,1:3,:])
    elif method == "Raw":
        return(bandpassData)
    elif method == 'BatchPPFsep':
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        phaseData = np.array([np.unwrap(np.angle(hilbert(i))) for i in bandpassData])
        powerData = np.array([np.abs(hilbert(i)) for i in bandpassData])
        data = []
        for i in range(8):
            for j in range(4):
                data.append(KaiserData[i][j])
            data.append(phaseData[i])
            data.append(powerData[i])
        return data
    else:
        raise "InvalidMethod"
}

SyntaxError: invalid syntax (<ipython-input-44-746e9c2e44b2>, line 1)

In [ ]:
from pylsl import StreamInlet, resolve_stream
import socket
import Preprocessing as pre
import numpy as np
from sklearn.externals import joblib
import pickle
import random
import os
import time
import scipy as sp
from scipy.signal import butter,freqz,lfilter,hilbert
#########Develop Blink_seq&Blink_string##################
v = [1,2,3,4,5,6]
Blink_seq = []
trials = 3
for j in range(trials):
    for i in random.sample(v, len(v)):
        Blink_seq.append(i)
        Blink_seq.append(0)

Blink_string = ''
for Blink_number in Blink_seq:
    Blink_string = Blink_string+str(Blink_number)+','
Blink_string = Blink_string[:-1] #take the last ',' out
#########Develop Blink_seq&Blink_string##################



# first resolve an EEG stream on the lab network
print("looking for an EEG stream...")
print("No error")
streams = resolve_stream('type', 'EEG')
print("founded")
# create a new inlet to read from the stream
inlet = StreamInlet(streams[0])
print(1)
sam_num = 0
Series = []
bb, a = pre.butter_bandpass(0.5, 30, 500, order=5)


#########Import model##################
svc_clf = joblib.load("weight2/weight_PavarisaNewUseRealSGD_1000_BatchPhasePower52.sav")
print(svc_clf)
P300_clf = joblib.load(open('finalized_p300model.sav', 'rb'))
#########Import model##################
print('models imported')

print('ready')
x=input("INSERT OK TO Start ")

print('starting')
#########Collecting Speech Imagery Data##################
while x == 'OK':
    
    sample, timestamp = inlet.pull_sample()
    Series.append(sample)
    sam_num = sam_num+1
    if sam_num == 1000:
        break
Series = np.asarray(Series).T
print(np.asarray(Series).shape)
#########Collecting Speech Imagery Data##################



#########Preprocess Data##################
bandpassData = pre.lfilter(bb, a, Series)
print(bandpassData.shape)
KaiserData = []
for i in range(8):
    tmp = pre.KaiserFil(bandpassData[i])
    KaiserData.append(tmp)
#FeaturedData = multibandfeat(KaiserData)
#print(np.asarray(KaiserData).shape)
#print("FeaturedData",len(FeaturedData))
#KaiserData = np.array(KaiserData)
phaseData = np.array([np.unwrap(np.angle(hilbert(i))) for i in bandpassData])
powerData = np.array([np.abs(hilbert(i)) for i in bandpassData])
print(phaseData.shape)
print(powerData.shape)
aaa=np.ravel((phaseData,powerData))
print("aaa",aaa.shape)
#shape=(8,4,1000)
#########Preprocess Data##################
A = np.reshape(aaa, (1, -1))
print("A",A.shape)
#########Predict for SI_Result##################
Seq = []
output = svc_clf.decision_function(A)#np.array([FeaturedData]))[0]
print("output",output.shape)
print(output)
for j in range(26):
    Seq.append([-output[0][j],j])#กำกับ percentage ด้วย Alphabet_index
Seq.sort() #sort percentage
SI_result = ''
for t in Seq[0:9]: #เลือก 9 ตัวที่มี percent มากสุด
        SI_result=SI_result+'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[t[1]] #แปรผลเป็นตัวอักษรimport socket
#########Predict for SI_Result##################


print('Result:'+SI_result)

#########Set up Server for Processing##################
sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock2.bind(("127.0.0.1", 5001))   
sock2.listen(1) #ต้องเช็คอีกรอบ

    #########Open Processing##################
os.chdir("application.windows64")
os.system("The_speller.exe")
os.chdir("../")
    #########Open Processing##################


conn2, addr2 = sock2.accept() #ต้องเช็คอีกรอบ
#########Set up Server for Processing##################

print('Connected by', addr2)
print('sending Result to Processing')
    
#########Send Result&Blink_string##################
arr = bytes(SI_result+Blink_string, 'utf-8')
conn2.sendall(arr)
#########Send Result&Blink_string##################

sample_num = 0
P300_data=[]
P300new=[]
#########Collecting Data while flashing##################
while True:
    data = str(repr(conn2.recv(1024)))[2:-1]
    print(data)
    if data=="Start" :
        print("ok")
        break

while True:
    sample, timestamp = inlet.pull_sample()
    P300_data.append(sample)
    #print("appended",sample)
    sample_num=sample_num+1
    time.sleep(0.004)
    if sample_num==465:
        break

#########Collecting Data while flashing##################
print(len(Blink_seq))
#########Predict P300_result##################
P300 = np.array(P300_data).T
print(type(P300),P300.shape)
new_P300 = []
for blink in range(18):
    tmps = []
    for node in range(8):
        #print(len(P300[node][50+20*blink:125+20*blink]),"ist",blink,node)
        tmps+=list(P300[node][50+20*blink:125+20*blink])
    #print(tmps)
    new_P300+=[tmps]
new_P300=np.array(new_P300)
#Data_shape = (18 blinks*8 nodes*3trials,75)
P300_result = P300_clf.decision_function(new_P300)
#########Predict P300_result##################

print(P300_result)

#########แปรผล##################
row_column = []
for i in range(18):
    if P300_result[i]<=0:
        row_column.append(Blink_seq[2*i])
print(len(row_column),'*')
for i in range(int(len(row_column)/2)):
    row = sorted(row_column[2*i:2*i+2])[0] #row will be 4-6
    column = sorted(row_column[2*i:2*i+2])[1] #column will be 1-3
    print(SI_result[(row-4)*3+column])
#########แปรผล##################

In [43]:
from pylsl import StreamInlet, resolve_stream
import socket
import Preprocessing as pre
import numpy as np
from sklearn.externals import joblib
import pickle
import random
import os
import time
import scipy as sp
from scipy.signal import butter,freqz,lfilter,hilbert
from sklearn import svm
train = []
label = []
#with open("weight2/weight_PavarisaNewUseRealSVC_1000_BatchPhasePower55.sav") as fin:
    #print(pickle.load(fin),'rb')
#svc_clf=joblib.load("weight2/weight_PavarisaNewUseRealSVC_1000_BatchPhasePower55.sav")
svc_clf=joblib.load("weight2/weight_PavarisaTestnoBaseLineSVC_1000_BatchPhasePower38.sav")
#svc_clf =  joblib.load("weight2/weight_PavarisaNewUseRealSVC_1000_BatchPhasePower55.sav","rb")
print(svc_clf)
for i in range(10):
    signal = np.random.uniform(size=96000)
    train.append(signal)
    label.append(random.randint(1,26))
print(np.array(train).shape)
print(train)
result = svc_clf.decision_function(train)
print(result)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
(10, 96000)
[array([0.91387639, 0.2883155 , 0.5142455 , ..., 0.19367209, 0.54441345,
       0.25688888]), array([0.44257236, 0.55833992, 0.53561963, ..., 0.71296826, 0.11963918,
       0.09888563]), array([0.06293265, 0.39177633, 0.95712972, ..., 0.68629029, 0.03887076,
       0.57195213]), array([0.88889227, 0.70725013, 0.08060103, ..., 0.88671155, 0.29816728,
       0.21421331]), array([0.36775377, 0.70315521, 0.83965557, ..., 0.58718175, 0.61273605,
       0.78192887]), array([0.33522256, 0.34797189, 0.31870414, ..., 0.60424119, 0.4005357 ,
       0.13844951]), array([0.69044242, 0.01324051, 0.33176532, ..., 0.06593932, 0.53918302,
       0.50380768]), array([0.61446772, 0.52296204, 0.0625275 , ..., 0.76172188, 0.43149372,
       0.62735413]), array([

In [ ]:
print(np.array([signal]).shape)

In [14]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
X, y = make_classification(n_features=4, random_state=0)
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X, y)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)
print(clf.coef_)
print(clf.intercept_)
print(clf.predict([[0, 0, 0, 0]]))
print(clf.decision_function([[0, 0, 0, 0],[1,1,1,1]]))

[[0.0855181  0.39414765 0.49848052 0.37514311]]
[0.28417574]
[1]
[0.28417574 1.63746511]


In [23]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X, y) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
print(clf.predict([[-0.8, -1]]))
print(clf.decision_function([[-0.8, -1],[1,2]]))

[1]
[-0.90840541  0.61336214]


In [ ]:
def preprocess(path,training=True,index = 0,method = "FrequenzySeperated"):
    if training:
        Rawdata1 = reader.readdata(path)[index][:1000] # file name inlet
        bb, a = butter_bandpass(0.5, 30, 250, order=5)
        bandpassData = lfilter(bb, a, Rawdata1)
    elif not training:
        bandpassData = path

    else:
        raise "ParameterError training invalid"

    KaiserData = []

    if method == "BatchPhasePower":
        phaseData = np.array([np.unwrap(np.angle(hilbert(i))) for i in bandpassData])
        powerData = np.array([np.abs(hilbert(i)) for i in bandpassData])
        return (phaseData,powerData)

    elif method == "FrequenzySeperated":
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        #print(KaiserData.shape)
        #plot8row(KaiserData,12)
        phaseData = np.array([np.array([ np.unwrap(np.angle(hilbert(i))) for i in j]) for j in KaiserData])
        powerData = np.array([np.array([ np.abs(hilbert(i)) for i in j]) for j in KaiserData])
        return (KaiserData,phaseData,powerData)
    elif method == "Alpha_theta":
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        #plot8row(KaiserData,12)'=
        return(KaiserData[:,1:3,:])
    elif method == "Raw":
        #print(KaiserData.shape)
        #plot8row(KaiserData,12)'=
        return(bandpassData)
    elif method == 'BatchPPFsep':
        for i in range(8):
            KaiserData.append(KaiserFil(bandpassData[i]))
        KaiserData = np.array(KaiserData)
        phaseData = np.array([np.unwrap(np.angle(hilbert(i))) for i in bandpassData])
        powerData = np.array([np.abs(hilbert(i)) for i in bandpassData])
        data = []
        for i in range(8):
            for j in range(4):
                data.append(KaiserData[i][j])
            data.append(phaseData[i])
            data.append(powerData[i])
        return data
    else:
        raise "InvalidMethod"